# Does everything to run entropy and creates slurm files to submit

In [1]:
!source activate py36

/usr/bin/sh: line 0: source: activate: file not found


In [2]:
import sys

#sys.path.append('/home/faske/g/anaconda3/envs/py34/lib/python3.4/site-packages')
sys.path.append('/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/data/gpfs/assoc/parchmanlab/tfaske/ipynb/include_utils")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
import re
from itertools import chain
#import Levenshtein as lv

In [3]:
root = '/data/gpfs/home/tfaske/d/rabbruh'

In [4]:
cd $root

/data/gpfs/assoc/denovo/tfaske/rabbruh


In [5]:
!mkdir entropy

In [8]:
ent_dir =os.path.join(root,'entropy')
assert ent_dir

In [20]:
cd $ent_dir

/data/gpfs/assoc/denovo/tfaske/rabbruh/entropy


In [13]:
!cp '../SNPcall/filtering/good_snps.recode.vcf.gz' .

In [22]:
!gunzip 'good_snps.recode.vcf.gz'

In [23]:
vcf = 'good_snps.recode.vcf'
vcf

'good_snps.recode.vcf'

In [24]:
perl_mpgl = '/data/gpfs/home/tfaske/g/src/perl_scripts/vcf2mpglV1.3TLP.pl'
perl_mean = '/data/gpfs/home/tfaske/g/src/perl_scripts/gl2genestV1.3.pl'

In [25]:
!perl $perl_mpgl $vcf

Number of loci: 30852; number of individuals 365


In [26]:
mpgl = 'good_snps.recode.mpgl'

In [27]:
!perl $perl_mean $mpgl mean

In [28]:
%load_ext rpy2.ipython

In [31]:
%%R
require(readr)
require(MASS)
require(LEA)
require(ggplot2)

source('/data/gpfs/home/tfaske/g/src/R/Imports.R')

setwd('/data/gpfs/home/tfaske/d/rabbruh/entropy')

R[write to console]: Loading required package: LEA



In [32]:
%%R
g <- read.table("pntest_mean_good_snps.recode.txt", header=F)
dim(g)

[1] 30852   365


In [33]:
%%R
Pop_ID <- read.csv("Pop_ID.csv")
pca_df <- PCA_entropy(t(g))

    PC1     PC2     PC3     PC4     PC5 
0.02336 0.01421 0.01185 0.01096 0.00961 


In [34]:
%%R
#######################################################################################
                                    #### LDA ####
#######################################################################################

k2<-kmeans(pca_df[,1:5],2,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k3<-kmeans(pca_df[,1:5],3,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k4<-kmeans(pca_df[,1:5],4,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k5<-kmeans(pca_df[,1:5],5,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k6<-kmeans(pca_df[,1:5],6,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k7<-kmeans(pca_df[,1:5],7,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k8<-kmeans(pca_df[,1:5],8,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k9<-kmeans(pca_df[,1:5],9,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k10<-kmeans(pca_df[,1:5],10,iter.max=10,nstart=10,algorithm="Hartigan-Wong")

ldak2<-lda(x=pca_df[,1:5],grouping=k2$cluster,CV=TRUE)
ldak3<-lda(x=pca_df[,1:5],grouping=k3$cluster,CV=TRUE)
ldak4<-lda(x=pca_df[,1:5],grouping=k4$cluster,CV=TRUE)
ldak5<-lda(x=pca_df[,1:5],grouping=k5$cluster,CV=TRUE)
ldak6<-lda(x=pca_df[,1:5],grouping=k6$cluster,CV=TRUE)
ldak7<-lda(x=pca_df[,1:5],grouping=k7$cluster,CV=TRUE)
ldak8<-lda(x=pca_df[,1:5],grouping=k8$cluster,CV=TRUE)
ldak9<-lda(x=pca_df[,1:5],grouping=k9$cluster,CV=TRUE)
ldak10<-lda(x=pca_df[,1:5],grouping=k10$cluster,CV=TRUE)

write.table(round(ldak2$posterior,5),file="ldak2.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak3$posterior,5),file="ldak3.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak4$posterior,5),file="ldak4.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak5$posterior,5),file="ldak5.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak6$posterior,5),file="ldak6.txt",quote=F,row.names=F,col.names=F)                                      
write.table(round(ldak7$posterior,5),file="ldak7.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak8$posterior,5),file="ldak8.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak9$posterior,5),file="ldak9.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak10$posterior,5),file="ldak10.txt",quote=F,row.names=F,col.names=F)

In [42]:
%%R
######### create entropy header ####

Pop_ID_list <- paste(Pop_ID$Pop,Pop_ID$ID,sep='_')

Header <- data.frame(dims = NA,Pop_ID_list)

dim(g)

[1] 30852   365


In [46]:
%%R
df <- t(Header)
dims <- paste(dim(g)[2],dim(g)[1],sep = " ")

df[1,1] <- dims
#head(df)
write.table(df,'entropy_header.txt',sep = " ",na ="",
            quote = FALSE,row.names = FALSE,col.names = FALSE)


In [47]:
!head entropy_header.txt

365 30852                                                                                                                                                                                                                                                                                                                                                                            
TF_100 TF_101 TF_102 TF_103 TF_104 TF_105 TF_106 TF_107 TF_108 TF_109 TF_10 TF_110 TF_111 TF_112 TF_113 TF_114 TF_115 TF_116 TF_117 TF_118 TF_119 TF_11 TF_120 TF_121 TF_122 TF_123 TF_124 TF_125 TF_126 TF_127 TF_128 TF_129 TF_12 TF_130 TF_131 TF_132 TF_133 TF_134 TF_135 TF_136 TF_137 TF_138 TF_139 TF_13 TF_140 TF_141 TF_142 TF_143 TF_144 TF_145 TF_146 TF_147 TF_148 TF_149 TF_14 TF_150 TF_151 TF_152 TF_153 TF_154 TF_155 TF_156 TF_157 TF_158 TF_159 TF_15 TF_160 TF_161 TF_162 TF_163 TF_164 TF_165 TF_166 TF_167 TF_168 TF_169 TF_16 TF_170 TF_171 TF_172 TF_173 TF_174 TF_175 TF_176 TF_177 TF_178 TF_179 TF_17 TF_180 TF_181 TF_1

In [48]:
!cat entropy_header.txt good_snps.recode.mpgl > entropy.mpgl

In [49]:
cd $ent_dir

/data/gpfs/assoc/denovo/tfaske/rabbruh/entropy


In [56]:
!mkdir shdir

In [60]:
### select slurm options
account = 'cpu-s1-bionres-0'
partition = 'cpu-s1-bionres-0'
time = '10-00:00:00' #time limit 4 days
cpus = 1
mem_cpu = 5000
email = 'tfaske@nevada.unr.edu'

#entropy settings
l = 100000
b = 30000
num_k = [2,3,4,5,6,7,8]
chains = 4

In [67]:
def write_entropy_sh(account,partition,time,cpus,mem_cpu,email,l,b,num_k,chains):
    for k in num_k:
        for c in range(1,chains+1):
            jobname = 'k%d_c%d' % (k,c)
            with open("shdir/run_entropy_%s.sh" % (jobname), "w") as o:
                o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name ent_%s
#SBATCH --output output_%s.txt
#SBATCH --mail-type=END
#SBATCH --mail-user=%s\n\n""" % (account,partition,time,cpus,mem_cpu,jobname,jobname,email))
        
                o.write("entropy -i ../entropy.mpgl -o ../entropy_%s.hdf5 -n 2 -l %d -b %d -t 10 -s 20 -e .01 -k %d -q ../ldak%d.txt -m 1 -w 0"
                        % (jobname,l,b,k,k))
    

In [68]:
write_entropy_sh(account,partition,time,cpus,mem_cpu,email,l,b,num_k,chains)

In [69]:
entShells = !find $ent_dir -name 'run_entropy*.sh'
entShells

['/data/gpfs/home/tfaske/d/rabbruh/entropy/shdir/run_entropy_k4_c2.sh',
 '/data/gpfs/home/tfaske/d/rabbruh/entropy/shdir/run_entropy_k5_c4.sh',
 '/data/gpfs/home/tfaske/d/rabbruh/entropy/shdir/run_entropy_k8_c4.sh',
 '/data/gpfs/home/tfaske/d/rabbruh/entropy/shdir/run_entropy_k6_c1.sh',
 '/data/gpfs/home/tfaske/d/rabbruh/entropy/shdir/run_entropy_k4_c4.sh',
 '/data/gpfs/home/tfaske/d/rabbruh/entropy/shdir/run_entropy_k8_c2.sh',
 '/data/gpfs/home/tfaske/d/rabbruh/entropy/shdir/run_entropy_k2_c1.sh',
 '/data/gpfs/home/tfaske/d/rabbruh/entropy/shdir/run_entropy_k3_c4.sh',
 '/data/gpfs/home/tfaske/d/rabbruh/entropy/shdir/run_entropy_k4_c3.sh',
 '/data/gpfs/home/tfaske/d/rabbruh/entropy/shdir/run_entropy_k2_c4.sh',
 '/data/gpfs/home/tfaske/d/rabbruh/entropy/shdir/run_entropy_k7_c4.sh',
 '/data/gpfs/home/tfaske/d/rabbruh/entropy/shdir/run_entropy_k2_c2.sh',
 '/data/gpfs/home/tfaske/d/rabbruh/entropy/shdir/run_entropy_k2_c3.sh',
 '/data/gpfs/home/tfaske/d/rabbruh/entropy/shdir/run_entropy_k8_

In [72]:
def write_sbatch_entShells(entShells):
         with open("shdir/sbatch_entropy.sh", "w") as o:
            o.write("""#!/usr/bin/env bash \n""")
            for sh in entShells:
                o.write("""sbatch %s\n""" % (sh))

In [73]:
write_sbatch_entShells(entShells)

# Run all entropy*sh
    cd /data/gpfs/home/tfaske/d/rabbruh/entropy/shdir
    source activate entropy
    bash sbatch_entropy.sh

# Use estpost to get DIC, q, and gprob

In [62]:
cd $ent_dir

/working/tfaske/piper/retic/entropy


In [67]:
hdf5_files = !find . -name '*hdf5'
hdf5_files = hdf5_files.sort()
hdf5_files

['./entropy_k2.hdf5',
 './entropy_k3.hdf5',
 './entropy_k4.hdf5',
 './entropy_k5.hdf5',
 './entropy_k6.hdf5']

In [73]:
!module load entropy/1.2

In [72]:
num_k = [2,3,4,5,6]

# nvm, figure out later. Run in terminal

In [71]:
#make DIC
for i in range(0,len(hdf5_files)):
    k = num_k[i]
    f = hdf5_files[i]
    dic = "DIC_k%d.txt" % (k)
    !estpost.entropy $f -s 3 -p deviance > $dic


/usr/bin/sh: estpost.entropy: command not found
/usr/bin/sh: estpost.entropy: command not found
/usr/bin/sh: estpost.entropy: command not found
/usr/bin/sh: estpost.entropy: command not found
/usr/bin/sh: estpost.entropy: command not found


In [77]:
dic_files = !find . -name 'DIC*'
dic_files

['./DIC_k2.txt',
 './DIC_k3.txt',
 './DIC_k4.txt',
 './DIC_k5.txt',
 './DIC_k6.txt']

In [79]:
for d in dic_files:
    !cat $d
    print('\n')

file = entropy_k2.hdf5
parameter dimensions for deviance: samples = 7000, chains = 1
Model deviance: 3770153.62
Effective number of parameters: 1724916.16
Model DIC: 5495069.78


file = entropy_k3.hdf5
parameter dimensions for deviance: samples = 7000, chains = 1
Model deviance: 3691381.34
Effective number of parameters: 1891442.59
Model DIC: 5582823.93


file = entropy_k4.hdf5
parameter dimensions for deviance: samples = 7000, chains = 1
Model deviance: 3609734.37
Effective number of parameters: 2097108.35
Model DIC: 5706842.72


file = entropy_k5.hdf5
parameter dimensions for deviance: samples = 7000, chains = 1
Model deviance: 3545331.26
Effective number of parameters: 2393933.78
Model DIC: 5939265.04


file = entropy_k6.hdf5
parameter dimensions for deviance: samples = 7000, chains = 1
Model deviance: 3416931.82
Effective number of parameters: 211269059.98
Model DIC: 214685991.80


